Look into twitter text data to predict if the given text has positive or negative sentiment
towards a particular brand. The dataset includes twitter text related to Apple and Google
products with user sentiment ranked between ‘positive’, ‘negative’, ‘neutral’ and ‘no_idea’,
sentiments. Create a simpleRNN or LSTM based classifiers to classify tweets into the four
classes. You can avoid ‘emotion_in_tweet_is_directed_at’ column.


#change runtime to gpu 

In [1]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 21 kB/s 
     |████████████████████████████████| 6.0 MB 56.5 MB/s 
     |████████████████████████████████| 439 kB 74.5 MB/s 
     |████████████████████████████████| 1.7 MB 68.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

#load libraries

In [1]:
import numpy as np
import pandas as pd

#load dataset

In [42]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',encoding = 'ISO-8859-1')

In [3]:
data.shape

(9093, 3)

In [ ]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [43]:
pd.set_option('display.max_colwidth', None)
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


In [44]:

tweet = data[['is_there_an_emotion_directed_at_a_brand_or_product','tweet_text']]
tweet.columns = ["label", "text"]
tweet.head()

,label,text
0,Negative emotion,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW."
1,Positive emotion,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW"
2,Positive emotion,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.
3,Negative emotion,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw
4,Positive emotion,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)"


In [ ]:
tweet.label.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: label, dtype: int64

In [6]:
tweet.label.unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

In [45]:
tweet['label']=tweet['label'].map({'Negative emotion':0,'Positive emotion':1,'No emotion toward brand or product':2,"I can't tell ":3})

<ipython-input-45-d977892227dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['label']=tweet['label'].map({'Negative emotion':0,'Positive emotion':1,'No emotion toward brand or product':2,"I can't tell ":3})


In [46]:
tweet.head(10)

,label,text
0,0.0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW."
1,1.0,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW"
2,1.0,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.
3,0.0,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw
4,1.0,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)"
5,2.0,@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd
6,2.0,NaN
7,1.0,"#SXSW is just starting, #CTIA is around the corner and #googleio is only a hop skip and a jump from there, good time to be an #android fan"
8,1.0,Beautifully smart and simple idea RT @madebymany @thenextweb wrote about our #hollergram iPad app for #sxsw! http://bit.ly/ieaVOB
9,1.0,Counting down the days to #sxsw plus strong Canadian dollar means stock up on Apple gear


In [8]:
tweet.shape

(9093, 2)

In [47]:
tweet.isnull().sum()

label    156
text       1
dtype: int64

In [48]:
tweet = tweet.dropna()

In [49]:
tweet.isnull().sum()

label    0
text     0
dtype: int64

In [50]:
from keras.preprocessing import text
tokenizer = text.Tokenizer() 
tokenizer.fit_on_texts(list(tweet['text']))
tokenized_texts = tokenizer.texts_to_sequences(tweet['text'])

In [51]:
tokenized_texts[0]

[5781,
 23,
 51,
 11,
 599,
 18,
 261,
 111,
 2548,
 625,
 6,
 1340,
 25,
 32,
 86,
 880,
 23,
 103,
 5,
 1103,
 2549,
 3897,
 6,
 1]

In [52]:
len(tokenized_texts[0])

24

In [53]:
from keras.utils import pad_sequences
X = pad_sequences(tokenized_texts, maxlen=100)

In [54]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 5781,
         23,   51,   11,  599,   18,  261,  111, 2548,  625,    6, 1340,
         25,   32,   86,  880,   23,  103,    5, 1103, 2549, 3897,    6,
          1], dtype=int32)

In [55]:
tokenizer.word_index

{'sxsw': 1,
 'mention': 2,
 'the': 3,
 'link': 4,
 'to': 5,
 'at': 6,
 'rt': 7,
 'for': 8,
 'ipad': 9,
 'google': 10,
 'a': 11,
 'apple': 12,
 'in': 13,
 'of': 14,
 'is': 15,
 'quot': 16,
 'and': 17,
 'iphone': 18,
 'store': 19,
 'on': 20,
 'up': 21,
 '2': 22,
 'i': 23,
 'new': 24,
 'austin': 25,
 'you': 26,
 'an': 27,
 'with': 28,
 'amp': 29,
 'app': 30,
 'my': 31,
 'it': 32,
 'social': 33,
 'launch': 34,
 'circles': 35,
 'this': 36,
 'android': 37,
 'pop': 38,
 'today': 39,
 'be': 40,
 'just': 41,
 'from': 42,
 'not': 43,
 'out': 44,
 'by': 45,
 'are': 46,
 'your': 47,
 'that': 48,
 'network': 49,
 'ipad2': 50,
 'have': 51,
 'via': 52,
 'will': 53,
 'line': 54,
 'about': 55,
 'free': 56,
 'get': 57,
 'now': 58,
 'if': 59,
 'called': 60,
 'me': 61,
 'mobile': 62,
 'party': 63,
 'so': 64,
 'sxswi': 65,
 'all': 66,
 'but': 67,
 'major': 68,
 'or': 69,
 'has': 70,
 'like': 71,
 'no': 72,
 "it's": 73,
 'time': 74,
 'one': 75,
 'what': 76,
 'temporary': 77,
 'w': 78,
 'opening': 79,
 'chec

In [56]:
len(tokenizer.word_index)

10044

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, tweet['label'].values, test_size=0.2)

In [58]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding,SimpleRNN,Dropout

In [59]:
model = Sequential()

model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim = 128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  

In [60]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1285760   
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                1390      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,287,751
Trainable params: 1,287,751
Non-

In [61]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
202/202 [==============================] - 19s 89ms/step - loss: -4.1627 - accuracy: 0.3224 - val_loss: -13.5512 - val_accuracy: 0.3580
Epoch 2/10
202/202 [==============================] - 19s 93ms/step - loss: -35.1050 - accuracy: 0.3291 - val_loss: -51.2610 - val_accuracy: 0.3427
Epoch 3/10
202/202 [==============================] - 18s 88ms/step - loss: -113.7904 - accuracy: 0.3286 - val_loss: -161.0078 - val_accuracy: 0.3594
Epoch 4/10
202/202 [==============================] - 18s 88ms/step - loss: -246.2644 - accuracy: 0.3311 - val_loss: -305.6280 - val_accuracy: 0.3594
Epoch 5/10
202/202 [==============================] - 18s 88ms/step - loss: -428.5972 - accuracy: 0.3320 - val_loss: -487.9353 - val_accuracy: 0.3594
Epoch 6/10
202/202 [==============================] - 18s 90ms/step - loss: -658.7017 - accuracy: 0.3362 - val_loss: -694.0323 - val_accuracy: 0.3594
Epoch 7/10
202/202 [==============================] - 18s 91ms/step - loss: -902.3337 - accuracy: 0.3414 

In [62]:
y_pred = model.predict(X_test)

56/56 [==============================] - 1s 13ms/step


In [66]:
y_pred[0]

array([1.], dtype=float32)

In [71]:
y_pred[y_pred>0.5] = 1
y_pred[y_pred<=0.5] = 0

In [72]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.33053691275167785

In [73]:
model = Sequential()

In [74]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [78]:
test1 = "Free entry in 2 a wkly comp to win  tkts!!"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)

In [79]:
out = model.predict(test2)

1/1 [==============================] - 0s 14ms/step


In [80]:
out


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,   56, 1169,   13,   22,   11, 2322,    5,  114,
        5128]], dtype=int32)